In [76]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [77]:
data = pd.read_csv('train-data.csv')
#data.columns = ['Positon X', '-']

In [78]:
data.head()

,Position X
0,-5247.64
1,-5246.81
2,-5244.54
3,-5240.81
4,-5235.62


In [79]:
data.dropna(inplace = True, axis = 1)

In [80]:
data.head()

,Position X
0,-5247.64
1,-5246.81
2,-5244.54
3,-5240.81
4,-5235.62


In [81]:
data.shape

(598, 1)

In [82]:
data.drop_duplicates(inplace= True)

In [83]:
data.head()

,Position X
0,-5247.64
1,-5246.81
2,-5244.54
3,-5240.81
4,-5235.62


In [84]:
data.to_csv('positions_wihtout_duplicates.csv', index = False)

In [85]:
data.shape

(598, 1)

In [86]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [87]:
last_positions = []
real_position = []
for i in range(10, 598):
    last_positions.append(data_scaled[i-10:i, 0])
    real_position.append(data_scaled[i, 0])

In [88]:
last_positions, real_position = np.array(last_positions), np.array(real_position)

In [89]:
last_positions = np.reshape(last_positions, (last_positions.shape[0], last_positions.shape[1], 1))

In [90]:
model = Sequential()
model.add(LSTM(units = 100, return_sequences = True, input_shape = (last_positions.shape[1], 1)))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 50))
model.add(Dropout(0.3))

model.add(Dense(units = 1, activation = 'linear'))
model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

In [91]:
t0 = time.time()
model.fit(last_positions, real_position, epochs= 100, batch_size = 32)
t1 = time.time()
delta = t1 - t0
delta = round(delta, 2)
print(f'O tempo de treino foi de: {delta}')

Epoch 1/100
19/19 [==============================] - 0s 7ms/step - loss: 0.2005 - mean_absolute_error: 0.2888
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0409 - mean_absolute_error: 0.1481
Epoch 3/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0377 - mean_absolute_error: 0.1460
Epoch 4/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0386 - mean_absolute_error: 0.1509
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0371 - mean_absolute_error: 0.1385
Epoch 6/100
19/19 [==============================] - 0s 8ms/step - loss: 0.0351 - mean_absolute_error: 0.1379
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0385 - mean_absolute_error: 0.1431
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0315 - mean_absolute_error: 0.1285
Epoch 9/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0311 - mean_absolute_error: 0.1285
Epoch 10/1

In [92]:
data_test = pd.read_csv('test_data.csv')

In [135]:
data_test.head()

,Position X
0,4761.94
1,4777.94
2,4793.94
3,4809.94
4,4825.94


In [136]:
complete_base = pd.concat((data, data_test), axis = 0)

In [137]:
complete_base.shape

(633, 1)

In [138]:
inputs = complete_base[len(complete_base) - len(data_test) - 10:].values

In [139]:
inputs = inputs.reshape(-1,1)

In [140]:
inputs = scaler.transform(inputs)

In [141]:
inputs.shape

(45, 1)

In [160]:
x_test = []
for i in range(10, 45):
    x_test.append(inputs[i - 10:i, 0])
x_test = np.array(x_test)
x_test.shape

(35, 10)

In [161]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [162]:
t0 = time.time()
predictions = model.predict(x_test)
t1 = time.time()
delta = t1 - t0
delta = round(delta, 2)
print(f'O tempo para prever 35 posições é: {delta}')

O tempo para prever 35 posições é: 0.11


In [163]:
predictions = scaler.inverse_transform(predictions)

In [180]:
predictions

array([[4728.394 ],
       [4732.134 ],
       [4735.8037],
       [4739.4043],
       [4742.938 ],
       [4746.4062],
       [4749.8105],
       [4753.15  ],
       [4756.4224],
       [4759.635 ],
       [4762.819 ],
       [4766.0327],
       [4769.3345],
       [4772.7593],
       [4776.3057],
       [4779.9185],
       [4783.4746],
       [4786.2246],
       [4788.9287],
       [4791.5864],
       [4794.201 ],
       [4796.7725],
       [4799.3   ],
       [4801.787 ],
       [4804.2334],
       [4806.64  ],
       [4809.0083],
       [4811.3384],
       [4813.631 ],
       [4815.8877],
       [4818.1094],
       [4820.296 ],
       [4822.4507],
       [4824.5786],
       [4826.6865]], dtype=float32)

In [181]:
data_test.head()

,Position X
0,4761.94
1,4777.94
2,4793.94
3,4809.94
4,4825.94


In [182]:
data_test.shape

(35, 1)

In [183]:
predictions.shape

(35, 1)

In [184]:
data2compare = data_test.copy()

In [185]:
data2compare.shape

(35, 1)

In [186]:
def roundPredict(value):
    return round(value,2)
data2compare['Predictions'] = predictions
data2compare['Predictions'] = data2compare['Predictions'].apply(roundPredict)

In [187]:
data2compare.head()

,Position X,Predictions
0,4761.94,4728.39
1,4777.94,4732.13
2,4793.94,4735.80
3,4809.94,4739.40
4,4825.94,4742.94
